# Production Model and Insights - Garage Crafters

In [386]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer

import statsmodels.api as sm

## Load clean training and test data

In [387]:
train = pd.read_csv('../data/cleaned_data/train_clean.csv')
test = pd.read_csv('../data/cleaned_data/test_clean.csv')

## Predictors and target Variables

In [388]:
predictors = [
'Garage Area',
'Garage Cars',
'Garage Cond',
'Garage Finish',
'Garage Qual',
'Garage Type',
'Garage Yr Blt',
'Gr Liv Area',
'Neighborhood',
]

In [389]:
X = train[predictors]
y = train['SalePrice']

## Split data into train and test sets

In [390]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 21)

In [391]:
X_train.shape, y_test.shape

((1640, 9), (411,))

## Column Transformer

In [392]:
categorical_cols = train[predictors].select_dtypes(exclude=['number']).columns.tolist()

In [393]:
categorical_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')

In [394]:
ct = make_column_transformer(
    (categorical_transformer, categorical_cols),
    remainder='passthrough'
)

## Transform training and testing data

In [395]:
X_train_ct = ct.fit_transform(X_train)

/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [396]:
X_test_ct = ct.transform(X_test)

In [397]:
column_names = ct.get_feature_names_out()

In [398]:
column_names

array(['onehotencoder__Garage Cond_Fa', 'onehotencoder__Garage Cond_Gd',
       'onehotencoder__Garage Cond_Po', 'onehotencoder__Garage Cond_TA',
       'onehotencoder__Garage Cond_no_garage',
       'onehotencoder__Garage Finish_RFn',
       'onehotencoder__Garage Finish_Unf',
       'onehotencoder__Garage Finish_no_garage',
       'onehotencoder__Garage Qual_Fa', 'onehotencoder__Garage Qual_Gd',
       'onehotencoder__Garage Qual_Po', 'onehotencoder__Garage Qual_TA',
       'onehotencoder__Garage Qual_no_garage',
       'onehotencoder__Garage Type_Attchd',
       'onehotencoder__Garage Type_Basment',
       'onehotencoder__Garage Type_BuiltIn',
       'onehotencoder__Garage Type_CarPort',
       'onehotencoder__Garage Type_Detchd',
       'onehotencoder__Garage Type_no_garage',
       'onehotencoder__Neighborhood_Blueste',
       'onehotencoder__Neighborhood_BrDale',
       'onehotencoder__Neighborhood_BrkSide',
       'onehotencoder__Neighborhood_ClearCr',
       'onehotencoder__Nei

In [399]:
X_train_ct_df = pd.DataFrame(X_train_ct, columns=column_names)

## Drop 'no_garage' Dummy Columns

In [400]:
# This code written with assistance from ChatGPT
no_garage_columns = [col for col in X_train_ct_df.columns if 'no_garage' in col]

if no_garage_columns:
    X_train_ct_df.drop(columns=no_garage_columns, inplace=True)
else:
    categorical_cols = train[predictors].select_dtypes(exclude=['number']).columns.tolist()
    first_columns_to_drop = [f"{col}_1" for col in categorical_cols if f"{col}_1" in X_train_ct_df.columns]
    X_train_ct_df.drop(columns=first_columns_to_drop, inplace=True)


In [401]:
no_garage_columns = [col for col in X_train_ct_df.columns if 'no_garage' in col]

In [402]:
X_train_ct_df.drop(columns=no_garage_columns, inplace=True)

## Initialize and train Linear Regression Model

In [403]:
model = LinearRegression()
model.fit(X_train_ct, y_train)

LinearRegression()

## Evaluate Model

In [404]:
# scoring model on training set
model.score(X_train_ct, y_train)

0.8009134303214196

In [405]:
#scoring model on testing set
model.score(X_test_ct, y_test)

0.7393897169564246

In [406]:
np.mean(cross_val_score(model, X_test_ct, y_test, cv=5))

-1.2610089418255112e+17

In [407]:
# setting baseline
y_pred_baseline = [np.mean(y_train)] * len(y_test)

In [408]:
# scoring mean baseline on testing set
r2_score(y_test, y_pred_baseline)

-0.014887741251467856

## Transform and Make Predictions on Unseen Testing Set

In [409]:
test_features = test[predictors]
test_features_ct = ct.transform(test_features)

In [410]:
predictions = model.predict(test_features_ct)

In [411]:
predictions_df = pd.DataFrame({'Id': test['Id'], 'SalePrice': predictions})

## OLS

In [412]:
y_train.reset_index(drop=True, inplace=True)
X_train_ct_df.reset_index(drop=True, inplace=True)

In [413]:
X_train_ct_df = sm.add_constant(X_train_ct_df)

In [414]:
ols = sm.OLS(y_train, X_train_ct_df).fit()

In [415]:
ols_summary = ols.summary()

In [416]:
ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                     139.3
Date:                Fri, 06 Oct 2023   Prob (F-statistic):               0.00
Time:                        06:50:19   Log-Likelihood:                -19533.
No. Observations:                1640   AIC:                         3.916e+04
Df Residuals:                    1593   BIC:                         3.941e+04
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                5.423e+04   1.02e+04      5.317      0.000    3.42e+04    7.42e+04
onehotencoder__Garage Cond_Fa        1.545e+05    4.6e+04      3.356      0.001    6.42e+04    2.45e+05
onehotencoder__Garage Cond_Gd        1.477e+05   4.78e+04      3.089      0.002    5.39e+04    2.41e+05
onehotencoder__Garage Cond_Po        1.381e+05   4.83e+04      2.862      0.004    4.35e+04    2.33e+05
onehotencoder__Garage Cond_TA         1.62e+05   4.54e+04      3.570      0.000     7.3e+04    2.51e+05
onehotencoder__Garage Finish_RFn    -1.892e+04   2683.528     -7.050      0.000   -2.42e+04   -1.37e+04
onehotencoder__Garage Finish_Unf    -2.106e+04   3151.191     -6.683      0.000   -2.72e+04   -1.49e+04
onehotencoder__Garage Qual_Fa       -1.827e+05   3.74e+04     -4.885      0.000   -2.56e+05   -1.09e+05
onehotencoder__Garage Qual_Gd        -1.45e+05   3.91e+04     -3.710      0.000   -2.22e+05   -6.83e+04
onehotencoder__Garage Qual_Po       -1.939e+05   5.48e+04     -3.541      0.000   -3.01e+05   -8.65e+04
onehotencoder__Garage Qual_TA       -1.804e+05   3.71e+04     -4.868      0.000   -2.53e+05   -1.08e+05
onehotencoder__Garage Type_Attchd        6e+04   9599.862      6.250      0.000    4.12e+04    7.88e+04
onehotencoder__Garage Type_Basment   3.771e+04   1.23e+04      3.068      0.002    1.36e+04    6.18e+04
onehotencoder__Garage Type_BuiltIn   5.055e+04   1.04e+04      4.864      0.000    3.02e+04    7.09e+04
onehotencoder__Garage Type_CarPort   1.305e+04   1.78e+04      0.733      0.463   -2.19e+04     4.8e+04
onehotencoder__Garage Type_Detchd    4.254e+04   9676.918      4.396      0.000    2.36e+04    6.15e+04
onehotencoder__Neighborhood_Blueste -2.917e+04   1.84e+04     -1.583      0.114   -6.53e+04    6982.838
onehotencoder__Neighborhood_BrDale  -1.484e+04    1.3e+04     -1.141      0.254   -4.03e+04    1.07e+04
onehotencoder__Neighborhood_BrkSide -1.113e+04   1.03e+04     -1.078      0.281   -3.14e+04    9130.266
onehotencoder__Neighborhood_ClearCr  9709.9993   1.16e+04      0.835      0.404   -1.31e+04    3.25e+04
onehotencoder__Neighborhood_CollgCr  8562.6827   9197.046      0.931      0.352   -9476.902    2.66e+04
onehotencoder__Neighborhood_Crawfor  2.071e+04   1.01e+04      2.052      0.040     913.911    4.05e+04
onehotencoder__Neighborhood_Edwards -1.697e+04   9623.734     -1.763      0.078   -3.58e+04    1909.086
onehotencoder__Neighborhood_Gilbert -9937.1526   9376.489     -1.060      0.289   -2.83e+04    8454.402
onehotencoder__Neighborhood_Greens    669.5139   2.27e+04      0.029      0.976   -4.39e+04    4.52e+04
onehotencoder__Neighborhood_GrnHill  1.125e+05   2.73e+04      4.119      0.000    5.89e+04    1.66e+05
onehotencoder__Neighborhood_IDOTRR  -3.897e+04   1.07e+04     -3.63

In [417]:
#This cell written with the help of ChatGPT
ols_summary_df = pd.read_html(ols_summary.tables[1].as_html(), header=0, index_col=0)[0]

In [418]:
ols_summary_df.head(60)

,coef,std err,t,P>|t|,[0.025,0.975]
const,54230.0000,10200.000,5.317,0.000,34200.000,74200.000
onehotencoder__Garage Cond_Fa,154500.0000,46000.000,3.356,0.001,64200.000,245000.000
onehotencoder__Garage Cond_Gd,147700.0000,47800.000,3.089,0.002,53900.000,241000.000
onehotencoder__Garage Cond_Po,138100.0000,48300.000,2.862,0.004,43500.000,233000.000
onehotencoder__Garage Cond_TA,162000.0000,45400.000,3.570,0.000,73000.000,251000.000
onehotencoder__Garage Finish_RFn,-18920.0000,2683.528,-7.050,0.000,-24200.000,-13700.000
onehotencoder__Garage Finish_Unf,-21060.0000,3151.191,-6.683,0.000,-27200.000,-14900.000
onehotencoder__Garage Qual_Fa,-182700.0000,37400.000,-4.885,0.000,-256000.000,-109000.000
onehotencoder__Garage Qual_Gd,-145000.0000,39100.000,-3.710,0.000,-222000.000,-68300.000
onehotencoder__Garage Qual_Po,-193900.0000,54800.000,-3.541,0.000,-301000.000,-86500.000


In [419]:
ols_summary_df.sort_values(by='coef', ascending=False).head(60)

,coef,std err,t,P>|t|,[0.025,0.975]
onehotencoder__Garage Cond_TA,162000.0000,45400.000,3.570,0.000,73000.000,251000.000
onehotencoder__Garage Cond_Fa,154500.0000,46000.000,3.356,0.001,64200.000,245000.000
onehotencoder__Garage Cond_Gd,147700.0000,47800.000,3.089,0.002,53900.000,241000.000
onehotencoder__Garage Cond_Po,138100.0000,48300.000,2.862,0.004,43500.000,233000.000
onehotencoder__Neighborhood_GrnHill,112500.0000,27300.000,4.119,0.000,58900.000,166000.000
onehotencoder__Neighborhood_StoneBr,95450.0000,10800.000,8.849,0.000,74300.000,117000.000
onehotencoder__Neighborhood_NridgHt,82230.0000,9498.121,8.657,0.000,63600.000,101000.000
onehotencoder__Garage Type_Attchd,60000.0000,9599.862,6.250,0.000,41200.000,78800.000
const,54230.0000,10200.000,5.317,0.000,34200.000,74200.000
onehotencoder__Garage Type_BuiltIn,50550.0000,10400.000,4.864,0.000,30200.000,70900.000
